In [1]:
# set working directory to parent
import os
os.chdir("..")

In [2]:
# import packages
import pandas as pd
import numpy as np

# tensorflow
import tensorflow as tf
import tensorflow_hub as hub

# user modules
from src.data import process_data 
from src.models.architecture import ModelBuilder
from src.models.compile import ModelCompiler
from src.models.train import ModelTrainer

# check tf version
print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")


Version:  2.0.0
Eager mode:  True
Hub version:  0.7.0
GPU is NOT AVAILABLE


In [3]:
# load train.csv  
# split into 1) training, 2) test/validation_data
train_data,test_data = process_data.train_split(process_data.retrieve_training(bucket = "quora-questions", file_name = "data/train.csv"))

Examples:
    Total: 1306122
    Positive: 80810 (6.19% of total)



In [ ]:
# architecture and hypter parameter input
# one experiment notebook per model type

In [4]:
# build transfer learnig model 
model_builder = ModelBuilder()
transfer_gnews = model_builder.transfer_simple(embedding_layer = "gnews", num_mid_nodes = 16, model_name = "mriv_model0")

In [5]:
# model summary
transfer_gnews.summary()

Model: "mriv_model0"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_text (InputLayer)      [(None,)]                 0         
_________________________________________________________________
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 353
Non-trainable params: 400,020
_________________________________________________________________


In [6]:
# compile model --> hyperparameters
model_compiler = ModelCompiler()
transfer_gnews = model_compiler.compile_model(transfer_gnews)


In [9]:
# train model 
model_trainer = ModelTrainer(model = transfer_gnews, experiment_name = "mriv_model0_exp0")
transfer_gnews_history = model_trainer.train_history(train_data = train_data, validation_data = test_data)


Epoch 1/20
2041/2041 [==============================] - 32s 16ms/step - loss: 0.1917 - tp: 9782.0000 - fp: 11647.0000 - tn: 968602.0000 - fn: 54866.0000 - accuracy: 0.9363 - precision: 0.4565 - recall: 0.1513 - auc: 0.8252 - val_loss: 0.0000e+00 - val_tp: 0.0000e+00 - val_fp: 0.0000e+00 - val_tn: 0.0000e+00 - val_fn: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.0000e+00
Epoch 2/20
2041/2041 [==============================] - 24s 12ms/step - loss: 0.1630 - tp: 11590.0000 - fp: 7626.0000 - tn: 972623.0000 - fn: 53058.0000 - accuracy: 0.9419 - precision: 0.6031 - recall: 0.1793 - auc: 0.8806 - val_loss: 0.1614 - val_tp: 2773.0000 - val_fp: 1753.0000 - val_tn: 243310.0000 - val_fn: 13389.0000 - val_accuracy: 0.9420 - val_precision: 0.6127 - val_recall: 0.1716 - val_auc: 0.8834
Epoch 3/20
2041/2041 [==============================] - 23s 11ms/step - loss: 0.1608 - tp: 12089.0000 - fp: 7818.0000 - tn: 972431.0000 - fn: 52559.0000 - ac

In [ ]:
# plot key metrics over training history 


In [11]:
# save serialized model

# hd5 file includes :
# The model's architecture
# The model's weight values (which were learned during training)
# The model's training config (what you passed to compile), if any
# The optimizer and its state, if any (this enables you to restart training where you left)

model_trainer.save_model()

In [12]:
model_trainer.model.reset_metrics()
model_trainer.model.save(os.getcwd() + '/data/saved_models/{}.h5'.format(model_trainer.experiment_name), save_format='tf')

In [ ]:
# test saved model and plot history 


array(<TensorSliceDataset shapes: ((), ()), types: (tf.string, tf.int64)>,
      dtype=object)